## ChatGPT Face Detector
Below are two programs about face recognition obtained from ChatGPT using various techniques of machine learning. 

### Background Context
##### K-Nearest Neighbors (KNN)
A popular supervised machine learning algorithm used in regression and classification. The model prediction is based on the similarities between the unseen data from the test set and its k nearest neighbors in the training set.<br>
- **Advantage:** simple and easy to implement <br>
- **Disadvantage:** lazy learner (train while making prediction) so slower and more costly (memory)

##### Convolutional Neural Network (CNN)
A type of deep learning neural network architecture used for image and speech processing. By using multiple interconnected layers, they can extract useful features from the input data and use them to make predictions<br>
- **Advantage:** multiple layers enable capture and recognize variations of data<br>
- **Disadvantage:** high complexity (expensive to train and use)<br>

### About the Dataset
The file contains two set of data: training and testing in the `train` and `val` folders respectively. Each of them contains two set of randomly chosen images displaying two different type of facial expressions: happy and sad. 

Data size: 
|  | train | test |
| --- | --- | --- |
| happy men | 25 | 5 |
| happy women | 25 | 5 |
| sad men | 25 | 5 |
| sad women | 25 | 5 |

[Image Source](https://stock.adobe.com/)

### Your tasks: 
KNN Model:
- Explain what does `Accuracy` tells you.
- Compute `precision` and `recall` and explain what they mean.
<br>

CNN Model:
- Train the model with the given dataset. What could do potentially improves `accuracy` of the model?
- Explain what does `loss` and `accuracy` tells you.
- Compute TP, TN, FP, FN
- Compute Precision and Recall
- How much of the True Positive were Male? Female?
- How much of the True Negative were Male? Female?
- Create a bar chart to show these proportion in terms of percentage.

### Format:
- For questions that require justification, include all your answers in a (one) Markdown cell after each program.
- For questions that require programming output, make sure it's clear what each output is. 

In [1]:
#use KNN
# Import necessary libraries
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the LFW dataset
# downloads and returns the images of people's face and labels
# only include people with at least 70 images in the dataset
lfw = datasets.fetch_lfw_people(min_faces_per_person=70)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(lfw.data, lfw.target, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors classifier
# n_neighbors: use the 5 nearest neighbors to make the predictions
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Calculate the accuracy of the classifier
# comparing the predicted labels (y_pred) with the true labels (y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print the accuracy of the classifier
print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# accuracy gives the proportion of accurately predicted sample from the total number of sample
# the closer to 1 the better


Accuracy: 0.5852713178294574
Precision: 0.5821667345464655
Recall: 0.5852713178294574


Accuracy tells us the ratio (unnormalized) of correctly predicted labels within the fed dataset. More specifically, after asking the model to predict on 'X_test,' it produces its dataframe of predictions into y_pred. The ratio of differences between y_pred and y_test dataframe (which is the frame that contains the properly labeled 'answers' or 'outputs') is then calculated by counting the differences between the two. In short, accuracy tells us the percentage of correct labels it got on the predicted test set.

The formula for Precision is as such: tp/(tp+fp) where tp and fp are true positives and false positives respectively. The precision tells us the ratio of correctly labeled positives to the total ratio of positives. Essentially, it judges the ability of the model to correctly identify a positive label. For example, precision can tell us the ratio between the correct amount of happy faces it detected versus the total amount of happy faces it detected. 

The formula for Recall is as such: tp/(tp+fn) where tp and fn are true positives and false negatives respectively. Recall tells us the amount of positives the model predicted correctly out of all positives that exist within the dataset. 


In [7]:
#use CNN
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directory paths for the training and validation datasets
train_dir = 'train'
val_dir = 'val'

# Define the number of classes (happy and sad)
num_classes = 2

# Define the input shape of the images
input_shape = (160, 160, 1)

# Define the batch size for the data generators
batch_size = 2

# Define the data generators for the training and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples//batch_size
)

# Save the model
model.save('face_classification_model.h5')

y_pred = model.predict(train_generator)
y_true = train_generator.classes

y_pred_binary = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_binary)

TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

print("\nTrue Positive (TP):", TP)
print("True Negative (TN):", TN)
print("False Positive (FP):", FP)
print("False Negative (FN):", FN)

Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/30
50/50 [==============================] - 1s 9ms/step - loss: 0.7566 - accuracy: 0.4200 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 2/30
50/50 [==============================] - 0s 6ms/step - loss: 0.6933 - accuracy: 0.4900 - val_loss: 0.6922 - val_accuracy: 0.5500
Epoch 3/30
50/50 [==============================] - 0s 6ms/step - loss: 0.7254 - accuracy: 0.5200 - val_loss: 0.8905 - val_accuracy: 0.5000
Epoch 4/30
50/50 [==============================] - 0s 6ms/step - loss: 0.6823 - accuracy: 0.5500 - val_loss: 0.8103 - val_accuracy: 0.4000
Epoch 5/30
50/50 [==============================] - 0s 6ms/step - loss: 0.6540 - accuracy: 0.5700 - val_loss: 0.6712 - val_accuracy: 0.5000
Epoch 6/30
50/50 [==============================] - 0s 6ms/step - loss: 0.6186 - accuracy: 0.6600 - val_loss: 0.6604 - val_accuracy: 0.5500
Epoch 7/30
50/50 [==============================] - 1s 12ms/step - loss: 0.6089

In [11]:
precision = precision_score(y_true, y_pred_binary, average='weighted')
recall = recall_score(y_true, y_pred_binary, average='weighted')
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.7467105263157895
Recall: 0.68


CNN Model:
- Train the model with the given dataset. What could do potentially improves `accuracy` of the model?
- Explain what does `loss` and `accuracy` tells you.
- Compute TP, TN, FP, FN
- Compute Precision and Recall
- How much of the True Positive were Male? Female?
- How much of the True Negative were Male? Female?
- Create a bar chart to show these proportion in terms of percentage.

In [12]:
y_pred_binary

array([0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

To improve the accuracy of the model, we can adjust the various hyperparameters such as batch_size and epochs. As you can see above, I lowered the batch_size, decreased the learning_rate, and increased the number of epochs.

Our loss function is cross-entropy which provides a loss value within the logarithmic scale. In general, loss will describe to us how well our model reflects the training data. It compares the predicted output versus the target output.
Accuracy is the ratio of correct predictions versus all predictions. 

True Positive (TP): 47
True Negative (TN): 21
False Positive (FP): 29
False Negative (FN): 3

Precision: 0.7467105263157895
Recall: 0.68



**Accuracy:** Proportion of correct prediction out of total prediction of the training/testing data.
- The higher the better
<br>

**Loss:** Measures the difference between the predicted and true output of the training/testing data. 
- The lower the better